# Overview

In notebook [GPT from scratch basic](https://www.kaggle.com/code/aisuko/gpt-from-scratch-basic) we build the neural net and training process with notebook style. Let's convert the code to a script.

# Import dependencies

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tensorboard import notebook
from torch.utils.tensorboard import SummaryWriter

%load_ext tensorboard

2024-07-27 03:54:31.814591: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 03:54:31.814725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 03:54:31.934857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Download training data

In [2]:
with open('/kaggle/input/gpt-from-scratch-basic/input.txt', 'r', encoding='utf-8') as f:
    text=f.read()

print(len(text))

1115394


# Define hyperparameters

In [3]:
batch_size=32 # how many independent sequences will we process in parallel?
block_size=8 # whay is the maximum contetx length for predictions?
max_iters=3000
eval_interval=300
learning_rate=1e-2
device='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters=200
writer=SummaryWriter()

torch.manual_seed(1337)

# Build vocabulary

In [4]:
# here are all the unique characters that occur in this text
chars=sorted(list(set(text)))
vocab_size=len(chars)

# create a mapping form characters to integers
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stoi[c] for c in s] # take a string, output a list of integers
decode=lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test split

In [5]:
data=torch.tensor(encode(text), dtype=torch.long)
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

# Get batch

In [6]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size, (batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y =x.to(device), y.to(device)
    return x,y

@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train', 'val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            logits, loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

# Define the model

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from the lookup table
        self.token_embedding_table=nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both(B,T) tensors of integers
        logits=self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss=None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T) # -1 it also ok
            loss=F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss=self(idx)
            # focus only on the last time step
            logits=logits[:,-1,:] # becomes (B,C)
            # apply softmax to get probabilities
            probs=F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next=torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx=torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [8]:
model=BigramLanguageModel(vocab_size)
m=model.to(device)

optimizer=torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training

In [9]:
for i in range(max_iters):
    # every once in a while evelate the loss on train and val sets
    if i % eval_interval==0:
        losses=estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        writer.add_scalar("Loss/train", losses['train'], i)
        
    # sample a batch of data
    xb, yb=get_batch('train')
    
    # evaluate the loss
    logits, loss=model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
writer.flush()
writer.close()

step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 2.8110, val loss 2.8249
step 600: train loss 2.5434, val loss 2.5682
step 900: train loss 2.4932, val loss 2.5088
step 1200: train loss 2.4863, val loss 2.5035
step 1500: train loss 2.4665, val loss 2.4921
step 1800: train loss 2.4683, val loss 2.4936
step 2100: train loss 2.4696, val loss 2.4846
step 2400: train loss 2.4638, val loss 2.4879
step 2700: train loss 2.4738, val loss 2.4911


# Generate from the model

In [10]:
%tensorboard --logdir log

In [11]:
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir log (started 0:00:00 ago; pid 61)


In [12]:
notebook.display(port=6006, height=1000)

Selecting TensorBoard with logdir log (started 0:00:00 ago; port 6006, pid 61).


In [13]:
context=torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))




CEThik brid owindakis b, bth

HAPet bobe d e.
S:
O:3 my d?
LUCous:
Wanthar u qur, t.
War dXENDoate awice my.

Hastarom oroup
Yowhthetof isth ble mil ndill, ath iree sengmin lat Heriliovets, and Win nghir.
Swanousel lind me l.
HAshe ce hiry:
Supr aisspllw y.
Hentofu n Boopetelaves
MPOLI s, d mothakleo Windo whth eisbyo the m dourive we higend t so mower; te

AN ad nterupt f s ar igr t m:

Thin maleronth,
Mad
RD:

WISo myrangoube!
KENob&y, wardsal thes ghesthinin couk ay aney IOUSts I&fr y ce.
J
